In [1]:
!pip install segmentation_models

     |████████████████████████████████| 51kB 3.4MB/s 


In [2]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [3]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import keras 

from keras.utils import normalize
from keras.metrics import MeanIoU


#Resizing images, if needed
SIZE_X = 244 
SIZE_Y = 244
n_classes=3 #Number of classes for segmentation

#Capture training image info as a list
train_images = []

Segmentation Models: using `tf.keras` framework.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
%cd  /content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Multiclass/

/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Multiclass


In [6]:
for directory_path in glob.glob("/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Multiclasstrain"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, 1)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        train_images.append(img)

In [7]:
train_images = np.array(train_images)

In [8]:
import os
from tqdm import tqdm 

TRAIN_PATH = "/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Multiclass/TestPictures"
train_ids = next(os.walk(TRAIN_PATH))[1]
path = TRAIN_PATH
train_masks = []
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
  for mask_file in next(os.walk(path + '/' + id_ +'/masks/'))[2]:
        mask = cv2.imread(path + '/' + id_ +'/masks/' + mask_file, 0)
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)
        train_masks.append(mask)
        
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

100%|██████████| 250/250 [02:49<00:00,  1.47it/s]


In [9]:
#Encode labels... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [12]:
train_images = np.expand_dims(train_images, axis=3)
#train_images = normalize(train_images, axis=1)

train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

#Create a subset of data for quick testing
#Picking 10% for testing and remaining for training
from sklearn.model_selection import train_test_split
X1, X_test, y1, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)

#Further split training data t a smaller subset for quick testing of models
X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.5, random_state = 0)

print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled

ValueError: ignored